In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_datareader.data as reader
import datetime as dt
import tensorflow as tf
import math
from keras.layers import LSTM, GRU, SimpleRNN
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix
pd.options.mode.chained_assignment = None  # default='warn'

In [ ]:
line = []

In [ ]:
f = open('/content/drive/MyDrive/2022-05-27','rb')
data = f.read()
data = data.decode('UTF-8')
#print((data))
data = data.split('\n')
print(len(data))

for j in range(1,len(data)):
  dat = data[j]
  str = dat.split(',')
  str = str[1:-2]
  line.append(str)
  print(str)
print(len(line))



76
['16275.8', '16311.8', '16274.2', '16308.1', '2022-05-27 03:50:00+00:00']
['16309.9', '16323.7', '16297.1', '16308.0', '2022-05-27 03:55:00+00:00']
['16309.0', '16319.8', '16288.8', '16304.5', '2022-05-27 04:00:00+00:00']
['16307.0', '16315.7', '16279.0', '16281.4', '2022-05-27 04:05:00+00:00']
['16280.4', '16316.65', '16277.0', '16308.7', '2022-05-27 04:10:00+00:00']
['16310.3', '16314.9', '16289.55', '16302.5', '2022-05-27 04:15:00+00:00']
['16300.0', '16313.85', '16287.15', '16297.0', '2022-05-27 04:20:00+00:00']
['16297.0', '16314.0', '16271.6', '16305.0', '2022-05-27 04:25:00+00:00']
['16304.0', '16305.5', '16254.45', '16255.0', '2022-05-27 04:30:00+00:00']
['16257.0', '16272.0', '16250.2', '16258.7', '2022-05-27 04:35:00+00:00']
['16258.7', '16286.05', '16258.65', '16273.8', '2022-05-27 04:40:00+00:00']
['16274.1', '16279.0', '16237.1', '16248.0', '2022-05-27 04:45:00+00:00']
['16246.5', '16262.25', '16244.1', '16246.8', '2022-05-27 04:50:00+00:00']
['16248.5', '16268.0', '162

In [ ]:
data_df = pd.DataFrame(line[0:-1], columns=['o','h','l','c','t'])
#data_df = data_df.drop(['r','e'],axis=1)
data_df['signal'] = 0
data_df

,o,h,l,c,t,signal
0,16053.5,16119.0,16053.45,16110.8,2022-05-20 03:50:00+00:00,0
1,16113.4,16124.2,16089.95,16100.9,2022-05-20 03:55:00+00:00,0
2,16101.2,16132.0,16101.25,16116.8,2022-05-20 04:00:00+00:00,0
3,16115.0,16119.2,16102.0,16116.5,2022-05-20 04:05:00+00:00,0
4,16117.8,16123.95,16100.2,16105.0,2022-05-20 04:10:00+00:00,0
...,...,...,...,...,...,...
444,16322.9,16325.0,16311.3,16319.3,2022-05-27 09:35:00+00:00,0
445,16320.3,16329.9,16313.55,16319.9,2022-05-27 09:40:00+00:00,0
446,16318.8,16335.0,16316.5,16332.5,2022-05-27 09:45:00+00:00,0
447,16332.2,16335.5,16328.0,16328.4,2022-05-27 09:50:00+00:00,0


In [ ]:
data_df.iloc[75]

o                           16228.6
h                           16295.0
l                           16209.0
c                           16264.2
t         2022-05-23 03:50:00+00:00
signal                            0
Name: 75, dtype: object

In [ ]:

data_df = data_df.astype({'o':float,'h':float,'l':float,'c':float})
data_df.dtypes

o         float64
h         float64
l         float64
c         float64
t          object
signal      int64
dtype: object

In [ ]:
data_df.iloc[75]

o                           16228.6
h                           16295.0
l                           16209.0
c                           16264.2
t         2022-05-23 03:50:00+00:00
signal                            0
Name: 75, dtype: object

In [ ]:
sl_var = 30.0
#data_df['signal'] = np.zeros(len(data_df))
for i in range(len(data_df)-43):
  if((float(data_df['c'].iloc[i])<float(data_df['c'].iloc[i+1])) and (float(data_df['c'].iloc[i+1])<float(data_df['c'].iloc[i+2])) and float(data_df['o'].iloc[i])<float(data_df['c'].iloc[i]) and float(data_df['o'].iloc[i+1])<float(data_df['c'].iloc[i+1]) and float(data_df['o'].iloc[i+2])<float(data_df['c'].iloc[i+2])):
    stop_loss = float(data_df['c'].iloc[i+2]) - sl_var
    break_loop = False
    #print(i)
    i=i+3
    for j in range(i,i+40):
      if(float(data_df['c'].iloc[j])<=stop_loss):
        #print(j)
        break_loop = True
        break
    if(break_loop == False):
      data_df.at[i,'signal'] = 1
    
  
#data_df.at[0,'signal'] = 1
for i in range(len(data_df)):
  if(int(data_df['signal'].iloc[i])==1):
    print(data_df['t'].iloc[i])


2022-05-20 07:40:00+00:00
2022-05-20 07:45:00+00:00
2022-05-20 07:50:00+00:00
2022-05-23 04:20:00+00:00
2022-05-23 04:25:00+00:00
2022-05-24 05:05:00+00:00
2022-05-26 06:25:00+00:00
2022-05-26 06:45:00+00:00
2022-05-26 07:15:00+00:00
2022-05-26 08:55:00+00:00
2022-05-26 09:20:00+00:00
2022-05-26 09:25:00+00:00
2022-05-26 09:55:00+00:00
None


In [ ]:
train_data_all = []
test_data_all = []
df = pd.DataFrame()
train_size = int(len(data_df)*0.8)
train_data = data_df.iloc[:train_size]
test_data = data_df.iloc[train_size:]

In [ ]:
def create_dataset (X, look_back = 3):
    Xs, ys = [], []
    X = X.drop(['h','l'],axis=1)
    for i in range(len(X)-look_back):
        v = X[X.columns[0:2]].iloc[i:i+look_back]
        Xs.append(v)
        ys.append(X['signal'].iloc[i+look_back])
 
    return np.array(Xs), np.array(ys)
LOOK_BACK = 3

X_train, y_train = create_dataset(train_data,LOOK_BACK)
X_test, y_test = create_dataset(test_data,LOOK_BACK)
y_train = y_train.reshape(-1,1)
y_test = y_test.reshape(-1,1)
  
# Print data shape
print('\nX_train.shape: ', X_train.shape)
print('y_train.shape: ', y_train.shape)
print('X_test.shape: ', X_test.shape) 
print('y_test.shape: ', y_test.shape)


X_train.shape:  (356, 3, 2)
y_train.shape:  (356, 1)
X_test.shape:  (87, 3, 2)
y_test.shape:  (87, 1)


In [ ]:
# Create LSTM model
def create_lstm(units):
    model = Sequential()
    # Input layer
    model.add(LSTM (units = units, return_sequences = True, 
                   input_shape = [X_train.shape[1], X_train.shape[2]]))
    model.add(Dropout(0.2)) 

    # Hidden layer
    model.add(LSTM(units = 100)) 
    model.add(Dropout(0.2))
    model.add(Dense(units = 100))
    model.add(Dense(2, activation='softmax')) 
    #Compile model
    #model.compile(optimizer='adam',loss='binary_crossentropy',metrics=["binary_accuracy"])
    model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
    return model
model_lstm = create_lstm(64)
model_lstm.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 3, 64)             17152     
                                                                 
 dropout_2 (Dropout)         (None, 3, 64)             0         
                                                                 
 lstm_3 (LSTM)               (None, 100)               66000     
                                                                 
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 93,454
Trainable params: 93,454
Non-trai

In [ ]:
def fit_model(model,X_train,y_train):
    #early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss',patience = 10)
    callbacks = [
      keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=10, min_lr=0.00000000000000000001
      ),
      keras.callbacks.EarlyStopping(monitor="val_loss", patience=100, verbose=1),
    ]
    history = model.fit(X_train, y_train, epochs = 10000,  
                        validation_split = 0.2,
                        batch_size = 16, shuffle = False, 
                        callbacks = callbacks)
    return history

In [ ]:
#For LSTM
models_lstm = []
histories_lstm = []
history_lstm = fit_model(model_lstm,X_train,y_train) #Call back that records events into history object

Epoch 1/10000


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


18/18 [==============================] - 4s 58ms/step - loss: nan - accuracy: 0.9542 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 2/10000
18/18 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.9789 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 3/10000
18/18 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.9789 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 4/10000
18/18 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.9789 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 5/10000
18/18 [==============================] - 0s 9ms/step - loss: nan - accuracy: 0.9789 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 6/10000
18/18 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.9789 - val_loss: nan - val_accuracy: 0.9583 - lr: 0.0010
Epoch 7/10000
18/18 [==============================] - 0s 10ms/step - loss: nan - accuracy: 0.9789 - val_loss:

In [ ]:
# Make prediction
def prediction(x_test, model):
    prediction = model.predict(x_test)
    prediction1 = []
    #sigmoid function is used in the last layer hence if output is greater than 0.5 then pred = 1 and vice versa
    #for i in range(len(prediction)):
     #if prediction[i]<0.5:
       #prediction[i]=1
     #else:
       #prediction[i]=0
    for i in range(len(prediction)):
       if prediction[i][0]>prediction[i][1]:
         prediction1.append(0)
       else:
         prediction1.append(1)

    return prediction1

# Plot test data vs prediction
def plot_conf_matrix(prediction, model_name, y_test):
    conf_matrix = confusion_matrix(y_true=y_test, y_pred=prediction)
    fig, ax = plt.subplots(figsize=(7.5, 7.5))
    ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
    for i in range(conf_matrix.shape[0]):
        for j in range(conf_matrix.shape[1]):
            ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
    plt.xlabel('Predictions', fontsize=18)
    plt.ylabel('Actuals', fontsize=18)
    plt.title('Confusion Matrix {model_name}'.format(model_name = model_name), fontsize=18)
    plt.show()  


In [ ]:
from sklearn.metrics import confusion_matrix


def compute_precision(y_test,pred):
	tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
	'''
	Precision = TP  / FP + TP 

	'''
	return (tp )/ float( tp + fp)
def compute_recall(y_test,pred):
	tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
	'''
	Recall = TP /FN + TP 

	'''
	return (tp  )/ float( tp + fn)
def compute_f1_score(y_test, pred):
    # calculates the F1 score
    precision = compute_precision(y_test, pred)
    recall = compute_recall(y_test, pred)
    f1_score = (2*precision*recall)/ (precision + recall)
    return f1_score

def compute_accuracy(y_test, pred):
		tn, fp, fn, tp = confusion_matrix(y_test, pred).ravel()
		return (tp+tn)/float(tp+tn+fp+fn)


In [ ]:
prediction_lstm= prediction(X_test,model_lstm)
precision = compute_precision(y_test, prediction_lstm)
recall = compute_recall(y_test, prediction_lstm)
accuracy = compute_accuracy(y_test, prediction_lstm)
f1 = compute_f1_score(y_test, prediction_lstm)

In [ ]:
print((precision))
print((recall))
print((accuracy))
print((f1))

#ANALYSIS: LOW PRECISION BECAUSE THE MODEL IS PREDICTING CONTINUOS 1'S DUE TO WHICH THERE ARE MANY FALSE POSITIVES AND ONE OF THEM COMES OUT TO BE TRUE POSITIVE HENCE LOW PRECISION
###########HIGH RECALL BECAUSE ALL THE ONES IN THE TESTING DATA ARE BEING CORRECTLY PREDICTED AS ONE HENCE THERE ARE 0 FALSE NEGATIVES 

0.04597701149425287
1.0
0.04597701149425287
0.08791208791208792
